## Модуль 5, домашнее задание
#### Автор: Вячеслав Барков

In [9]:
%load_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = 10

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
import json
import sqlalchemy
import pandas as pd

In [4]:
with open('.secret') as f:
    data = json.load(f)
    user = data['user']
    password = data['password']
    server = data['server']
    db = data['db']

In [5]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{server}/{db}')

In [6]:
%sql postgresql://$user:$password@$server/$db

## Задание 1.


Как мы видели по датасету shipment, не по всем городам в справочнике выполнялись поставки и не во всех городах у нас есть водители. Мы хотим написать запрос, который выдаст нам следующее:
- Название города.
- Фамилию и имя водителя, проживающего в городе,
- Дату поставки в этот город 
- Компанию-получателя груза.

Для отбора ФИО водителя мы хотим использовать подзапрос в секции SELECT.
В условиях мы хотим указать с помощью подзапроса, что:
- В данном городе проживает хотя бы один водитель ИЛИ
- В данный город была осуществлена хотя бы одна доставка.

**Решение:** Нам необходимо вывести имя любого водителя, если хотя бы один из водителей проживает в определееном городе, а также любую дату доставки и имя получателя, если хотя бы одна доставка осуществлялась в определнный город. Не будем выводить случайные строки по усмотрению самой БД, а выведем первого водителя по алфавиту и последнюю доставку по дате.

In [10]:
%%sql

SELECT
    c.city_name,
    (SELECT
        d.first_name || ' ' || d.last_name driver_name
    FROM
        shipping.driver d
    WHERE
        c.city_id = d.city_id
    ORDER BY
        d.last_name,
        d.first_name
    LIMIT 1
    ) driver_name,
    ss.ship_date,
    cc.cust_name 
FROM 
    shipping.city c
    LEFT JOIN
    (SELECT DISTINCT ON (s.city_id)
        s.city_id,
        s.ship_date,
        s.cust_id 
    FROM 
        shipping.shipment s
    ORDER BY
        s.city_id,
        s.ship_date DESC
    ) ss ON ss.city_id = c.city_id 
    LEFT JOIN shipping.customer cc ON cc.cust_id = ss.cust_id 
ORDER BY 
    c.city_name 

601 rows affected.


city_name,driver_name,ship_date,cust_name
Abilene,None,2017-06-15,Sunguard Window Tinting & Truck Accessories
Akron,None,2017-12-18,NW RV Supply
Alameda,None,2017-11-02,Marlin Ingram RV Center
Albany,None,None,None
Albany,None,None,None
Albuquerque,None,None,None
Alexandria,None,2017-11-30,Hawkins Trailer & Equipment CO
Alhambra,None,2017-03-11,Mid Town Rent-A-Car
Allentown,None,2017-12-04,NW RV Supply
Amarillo,None,2016-10-07,HYD Products


## Задание 2.


Мы хотим построить и объединить результаты двух запросов по книжному датасету. </br> Запрос 1. Книги жанра non-fiction, где у книги рейтинг менее или равен 4.00 
- Название книги
- Автор
- Издательство
- Рейтинг книги

Запрос 2. Книги жанра fiction (помните, что у нас два значения соответствуют этому классификатору?), где у книги рейтинг менее или равен 4.00, при этом у автора рейтинг novice или intermediate (т.е. мы не берем знаменитых писателей).
- Название книги
- Автор
- Издательство
- Рейтинг книги

In [11]:
%%sql

SET
    search_path
TO
    books;

Done.


[]

In [12]:
%%sql

SELECT 
    b.book_name,
    b.author,
    b.publisher,
    b.book_average_rating 
FROM 
    books b 
WHERE
    b.genre = 'nonfiction'
    AND b.book_average_rating <= 4.00

UNION ALL 

SELECT 
    b.book_name,
    b.author,
    b.publisher,
    b.book_average_rating 
FROM 
    books b 
WHERE
    b.genre IN ('genre fiction', 'fiction')
    AND b.book_average_rating <= 4.00
    AND b.author_rating IN ('Novice', 'Intermediate')

510 rows affected.


book_name,author,publisher,book_average_rating
Midnight in the Garden of Good and Evil,John Berendt,Hachette Book Group,3.9
Tess of the d'Urbervilles: A Pure Woman Faithfully Presented,"Thomas Hardy, Tim Dolin, Margaret R. Higonnet",Hachette Book Group,3.77
Tuck Everlasting,Natalie Babbitt,"Amazon Digital Services, Inc.",3.85
Christine,"Stephen King, Marie Milpois","Amazon Digital Services, Inc.",3.72
Baby Proof,Emily Giffin,Macmillan,3.71
Dreams from My Father,Barack Obama,HarperCollins Publishers,3.81
Congo,Michael Crichton,Random House LLC,3.55
Anansi Boys,Neil Gaiman,"Amazon Digital Services, Inc.",4.0
Walk Two Moons,Sharon Creech,"Amazon Digital Services, Inc.",3.95
"Julie & Julia: 365 Days, 524 Recipes, 1 Tiny Apartment Kitchen",Julie Powell,Random House LLC,3.67
